# Ejercicio de Laboratorio: Algoritmos Genéticos
## Problema: Planificación de Horarios de Cursos

### Información del Estudiante
- **Nombre:** ______________________________
- **Código:** ______________________________
- **Fecha:** ______________________________

---

## Objetivo del Ejercicio

Implementar un algoritmo genético para resolver un problema de **planificación de horarios de cursos**. Deberás completar tres funciones clave:

1. **Función de Aptitud (Fitness)** - Evaluar qué tan bueno es un horario
2. **Operador de Cruce (Crossover)** - Combinar dos horarios padres
3. **Operador de Mutación (Mutation)** - Introducir variación aleatoria

**Tiempo estimado:** 90-120 minutos

---

## Descripción del Problema

Un colegio necesita programar **8 cursos** en **5 bloques de tiempo** disponibles.

### Cursos Disponibles:
1. **Matemáticas (MAT)** - 45 estudiantes
2. **Física (FIS)** - 30 estudiantes  
3. **Química (QUI)** - 35 estudiantes
4. **Programación (PRG)** - 40 estudiantes
5. **Literatura (LIT)** - 25 estudiantes
6. **Historia (HIS)** - 30 estudiantes
7. **Inglés (ING)** - 35 estudiantes
8. **Educación Física (EDF)** - 50 estudiantes

### Bloques de Tiempo:
- **Bloque 1:** Lunes 8:00-10:00
- **Bloque 2:** Lunes 10:00-12:00
- **Bloque 3:** Miércoles 8:00-10:00
- **Bloque 4:** Miércoles 10:00-12:00
- **Bloque 5:** Viernes 8:00-10:00

### Salones Disponibles:
- **Salón A:** Capacidad 40 estudiantes
- **Salón B:** Capacidad 30 estudiantes
- **Salón C:** Capacidad 50 estudiantes

### Restricciones DURAS (DEBEN cumplirse):
1. **No solapamiento:** Un salón no puede tener dos cursos al mismo tiempo
2. **Capacidad:** El salón debe tener capacidad suficiente para los estudiantes
3. **Profesores compartidos:** Algunos cursos los dicta el mismo profesor:
   - MAT y FIS (mismo profesor)
   - QUI y PRG (mismo profesor)

### Restricciones BLANDAS (Preferibles):
1. **Horas tempranas:** Preferir bloques 1, 2, 3 sobre 4 y 5
2. **Uso eficiente:** Evitar salones muy grandes para grupos pequeños

---

## Representación de la Solución

Cada **individuo** (solución) se representa como una lista de tuplas:

```python
individuo = [
    (bloque_tiempo, salon),  # Para MAT
    (bloque_tiempo, salon),  # Para FIS
    (bloque_tiempo, salon),  # Para QUI
    (bloque_tiempo, salon),  # Para PRG
    (bloque_tiempo, salon),  # Para LIT
    (bloque_tiempo, salon),  # Para HIS
    (bloque_tiempo, salon),  # Para ING
    (bloque_tiempo, salon),  # Para EDF
]
```

**Ejemplo:**
```python
[(1, 'A'), (2, 'B'), (3, 'C'), (4, 'A'), (5, 'B'), (1, 'C'), (2, 'A'), (3, 'B')]
```

Significa:
- MAT: Bloque 1, Salón A
- FIS: Bloque 2, Salón B
- QUI: Bloque 3, Salón C
- etc.

---

## Configuración Inicial

Ejecuta esta celda para cargar los datos del problema:

In [ ]:
import random
from typing import List, Tuple

# Cursos: (nombre, número de estudiantes)
courses = [
    ('MAT', 45),  # 0
    ('FIS', 30),  # 1
    ('QUI', 35),  # 2
    ('PRG', 40),  # 3
    ('LIT', 25),  # 4
    ('HIS', 30),  # 5
    ('ING', 35),  # 6
    ('EDF', 50)   # 7
]

# Bloques de tiempo disponibles
time_blocks = [1, 2, 3, 4, 5]
block_names = {
    1: 'Lun 8-10',
    2: 'Lun 10-12',
    3: 'Mie 8-10',
    4: 'Mie 10-12',
    5: 'Vie 8-10'
}

# Salones: {nombre: capacidad}
rooms = {
    'A': 40,
    'B': 30,
    'C': 50
}

# Índices de cursos que comparten profesor (no pueden ser simultáneos)
same_professor = [
    (0, 1),  # MAT y FIS
    (2, 3)   # QUI y PRG
]

print("✓ Configuración cargada")
print(f"  Cursos: {len(courses)}")
print(f"  Bloques: {len(time_blocks)}")
print(f"  Salones: {len(rooms)}")

## Funciones Auxiliares (Proporcionadas)

Estas funciones te ayudarán a visualizar y crear individuos:

In [ ]:
def print_schedule(individual):
    """Imprime un horario de forma legible."""
    print("\n" + "="*60)
    print("HORARIO")
    print("="*60)
    for i, (block, room) in enumerate(individual):
        name, students = courses[i]
        print(f"{name:3s} | {block_names[block]:10s} | Salón {room} (Cap:{rooms[room]:2d}) | Est:{students:2d}")
    print("="*60)

def create_random_individual():
    """Crea un individuo aleatorio."""
    individual = []
    for i in range(len(courses)):
        block = random.choice(time_blocks)
        room = random.choice(list(rooms.keys()))
        individual.append((block, room))
    return individual

def create_population(size=20):
    """Crea una población inicial."""
    return [create_random_individual() for _ in range(size)]

# Probar
sample = create_random_individual()
print_schedule(sample)

---

# TAREA 1: Función de Aptitud (Fitness) [40 puntos]

## Instrucciones

Completa la función `fitness()` que evalúa qué tan bueno es un horario.

### Sistema de Puntuación:

**Puntaje Base:** 200 puntos

**Penalizaciones por restricciones DURAS:**
- **-50 puntos** por cada conflicto de salón (dos cursos mismo bloque y salón)
- **-50 puntos** por cada salón con capacidad insuficiente
- **-50 puntos** por cada par de cursos del mismo profesor en mismo bloque

**Penalizaciones por restricciones BLANDAS:**
- **-5 puntos** por cada curso en bloques tardíos (bloques 4 o 5)
- **-3 puntos** si un curso pequeño (<35 estudiantes) usa salón grande (>45 capacidad)

### Pseudocódigo:

```
función fitness(individual):
    score = 200
    
    # 1. Conflictos de salones
    para cada par de cursos (i, j) donde i < j:
        si tienen mismo bloque Y mismo salón:
            score -= 50
    
    # 2. Capacidad de salones
    para cada curso i:
        si estudiantes[i] > capacidad del salón:
            score -= 50
    
    # 3. Profesores compartidos
    para cada par en same_professor:
        si ambos en mismo bloque:
            score -= 50
    
    # 4. Bloques tardíos
    para cada curso:
        si bloque >= 4:
            score -= 5
    
    # 5. Desperdicio de espacio
    para cada curso:
        si estudiantes < 35 Y capacidad > 45:
            score -= 3
    
    retornar score
```

---

In [ ]:
def fitness(individual):
    """
    Calcula la aptitud de un horario.
    
    Args:
        individual: Lista de tuplas [(bloque, salón), ...] para cada curso
    
    Returns:
        score: Puntaje de aptitud (mayor es mejor)
    """
    # Puntaje base
    score = 200
    
    # ============================================================
    # TU CÓDIGO AQUÍ
    # ============================================================
    
    # 1. RESTRICCIÓN DURA: Conflictos de salones (-50 puntos cada uno)
    # Penalizar si dos cursos están en el mismo bloque y salón
    # Pista: usa dos bucles anidados con range(len(individual))
    # Compara individual[i] con individual[j] donde i < j
    
    
    
    
    # 2. RESTRICCIÓN DURA: Capacidad de salones (-50 puntos cada uno)
    # Penalizar si un curso tiene más estudiantes que la capacidad del salón
    # Pista: courses[i] = (nombre, estudiantes)
    #        individual[i] = (bloque, salón)
    #        rooms[salón] = capacidad
    
    
    
    
    # 3. RESTRICCIÓN DURA: Profesores compartidos (-50 puntos cada uno)
    # Penalizar si cursos del mismo profesor están en el mismo bloque
    # Pista: same_professor contiene tuplas (curso1_idx, curso2_idx)
    #        compara individual[curso1_idx][0] con individual[curso2_idx][0]
    
    
    
    
    # 4. RESTRICCIÓN BLANDA: Bloques tardíos (-5 puntos cada uno)
    # Penalizar cursos en bloques 4 o 5
    # Pista: individual[i][0] es el bloque del curso i
    
    
    
    
    # 5. RESTRICCIÓN BLANDA: Desperdicio de espacio (-3 puntos cada uno)
    # Penalizar si curso pequeño (<35 est.) usa salón grande (>45 cap.)
    # Pista: courses[i][1] es el número de estudiantes
    #        rooms[individual[i][1]] es la capacidad del salón
    
    
    
    
    # ============================================================
    # FIN DE TU CÓDIGO
    # ============================================================
    
    return score

# Probar tu función
test_ind = create_random_individual()
print_schedule(test_ind)
print(f"\nFitness: {fitness(test_ind)}")

---

# TAREA 2: Operador de Cruce (Crossover) [30 puntos]

## Instrucciones

Implementa un operador de **cruce de un punto** que combine dos horarios padres.

### Funcionamiento:
1. Selecciona un punto de corte aleatorio (entre 1 y 7)
2. Copia los primeros cursos del padre 1 (hasta el punto de corte)
3. Copia los cursos restantes del padre 2 (después del punto de corte)

### Ejemplo:
```python
Padre 1: [(1,'A'), (2,'B'), (3,'C'), (4,'A'), (5,'B'), (1,'C'), (2,'A'), (3,'B')]
Padre 2: [(2,'C'), (3,'A'), (4,'B'), (5,'C'), (1,'A'), (2,'B'), (3,'C'), (4,'A')]
Punto: 4

Hijo:    [(1,'A'), (2,'B'), (3,'C'), (4,'A'), (1,'A'), (2,'B'), (3,'C'), (4,'A')]
         |------------- P1 -------------|------------ P2 -------------|
```

### Pseudocódigo:
```
función crossover(padre1, padre2):
    punto = número aleatorio entre 1 y 7
    hijo = padre1[0:punto] + padre2[punto:]
    retornar hijo
```

---

In [ ]:
def crossover(parent1, parent2):
    """
    Crea un hijo combinando dos padres usando cruce de un punto.
    
    Args:
        parent1: Primer padre (horario)
        parent2: Segundo padre (horario)
    
    Returns:
        child: Nuevo horario combinando ambos padres
    """
    # ============================================================
    # TU CÓDIGO AQUÍ
    # ============================================================
    
    # 1. Seleccionar punto de corte aleatorio entre 1 y 7
    # Pista: usa random.randint(1, 7)
    
    
    # 2. Crear hijo combinando:
    #    - Primera parte de parent1 (índices 0 hasta punto)
    #    - Segunda parte de parent2 (índices punto hasta el final)
    # Pista: usa slicing de listas y concatenación (+)
    
    
    # 3. Retornar el hijo
    
    
    # ============================================================
    # FIN DE TU CÓDIGO
    # ============================================================
    
    pass  # Elimina este pass y descomenta: return child

# Probar tu operador
p1 = create_random_individual()
p2 = create_random_individual()

print("Padre 1:")
print_schedule(p1)
print(f"Fitness: {fitness(p1)}")

print("\nPadre 2:")
print_schedule(p2)
print(f"Fitness: {fitness(p2)}")

child = crossover(p1, p2)
print("\nHijo:")
print_schedule(child)
print(f"Fitness: {fitness(child)}")

---

# TAREA 3: Operador de Mutación [30 puntos]

## Instrucciones

Implementa un operador de mutación que introduzca cambios aleatorios.

### Funcionamiento:
Para cada curso, con probabilidad `prob` (20%):
- Decide aleatoriamente si cambiar:
  - Solo el bloque de tiempo
  - Solo el salón  
  - Ambos

### Ejemplo:
```python
Original: [(1,'A'), (2,'B'), (3,'C'), (4,'A'), ...]
Mutación en posición 1 (cambiar ambos):
Mutado:   [(1,'A'), (5,'C'), (3,'C'), (4,'A'), ...]
                     ^^^^^^^
```

### Pseudocódigo:
```
función mutate(individual, prob=0.2):
    para cada posición i:
        si random.random() < prob:
            tipo = elegir aleatoriamente entre [1, 2, 3]
            si tipo == 1:  # Cambiar solo bloque
                nuevo_bloque = elegir bloque aleatorio
                individual[i] = (nuevo_bloque, salón_actual)
            sino si tipo == 2:  # Cambiar solo salón
                nuevo_salón = elegir salón aleatorio
                individual[i] = (bloque_actual, nuevo_salón)
            sino:  # Cambiar ambos
                nuevo_bloque = elegir bloque aleatorio
                nuevo_salón = elegir salón aleatorio
                individual[i] = (nuevo_bloque, nuevo_salón)
    retornar individual
```

---

In [ ]:
def mutate(individual, prob=0.2):
    """
    Muta un horario cambiando aleatoriamente bloques o salones.
    
    Args:
        individual: Horario a mutar
        prob: Probabilidad de mutación por curso (default: 20%)
    
    Returns:
        individual: Horario mutado
    """
    # ============================================================
    # TU CÓDIGO AQUÍ
    # ============================================================
    
    # Recorrer cada curso
    for i in range(len(individual)):
        # Decidir si mutar este curso (probabilidad prob)
        if random.random() < prob:
            
            # Obtener valores actuales
            current_block, current_room = individual[i]
            
            # Elegir tipo de mutación: 1=solo bloque, 2=solo salón, 3=ambos
            mutation_type = random.choice([1, 2, 3])
            
            if mutation_type == 1:
                # Mutar solo el bloque de tiempo
                # Pista: new_block = random.choice(time_blocks)
                #        individual[i] = (new_block, current_room)
                pass
                
            elif mutation_type == 2:
                # Mutar solo el salón
                # Pista: new_room = random.choice(list(rooms.keys()))
                #        individual[i] = (current_block, new_room)
                pass
                
            else:
                # Mutar ambos
                # Pista: combina los dos anteriores
                pass
    
    # ============================================================
    # FIN DE TU CÓDIGO
    # ============================================================
    
    return individual

# Probar tu operador
original = create_random_individual()
print("Original:")
print_schedule(original)
print(f"Fitness: {fitness(original)}")

mutated = [x for x in original]  # Copia
mutated = mutate(mutated, prob=0.5)  # 50% para ver más mutaciones

print("\nMutado (prob=50%):")
print_schedule(mutated)
print(f"Fitness: {fitness(mutated)}")

---

# Función de Evolución (Proporcionada)

Esta función combina todos tus operadores:

In [ ]:
def selection(population):
    """Selección por torneo de 3 individuos."""
    tournament = random.sample(population, 3)
    return max(tournament, key=fitness)

def evolve(population, generations=50, verbose=True):
    """Ejecuta el algoritmo genético."""
    best_overall = None
    best_overall_fitness = float('-inf')
    
    for gen in range(generations):
        # Elitismo: mantener los 2 mejores
        sorted_pop = sorted(population, key=fitness, reverse=True)
        new_population = sorted_pop[:2]
        
        # Generar resto de población
        while len(new_population) < len(population):
            parent1 = selection(population)
            parent2 = selection(population)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)
        
        population = new_population
        
        # Mejor de esta generación
        best = max(population, key=fitness)
        best_fit = fitness(best)
        
        if best_fit > best_overall_fitness:
            best_overall = best
            best_overall_fitness = best_fit
        
        if verbose and (gen % 10 == 0 or gen == generations - 1):
            print(f"Gen {gen+1:3d}: Mejor fitness = {best_fit:6.1f}")
    
    return best_overall

print("✓ Funciones de evolución cargadas")

---

# Ejecutar el Algoritmo Genético

Una vez completadas las tres tareas, ejecuta esta celda:

In [ ]:
print("="*70)
print("INICIANDO ALGORITMO GENÉTICO")
print("="*70)

# Crear población inicial
initial_pop = create_population(size=30)

print(f"\nPoblación inicial: {len(initial_pop)} individuos")
print(f"Fitness promedio: {sum(fitness(ind) for ind in initial_pop) / len(initial_pop):.1f}")
print(f"Mejor fitness: {max(fitness(ind) for ind in initial_pop):.1f}")

print("\n" + "="*70)
print("EVOLUCIÓN")
print("="*70)

# Ejecutar
best_solution = evolve(initial_pop, generations=100)

print("\n" + "="*70)
print("MEJOR SOLUCIÓN")
print("="*70)
print_schedule(best_solution)
print(f"\nFitness final: {fitness(best_solution):.1f}")

# Análisis
print("\n" + "="*70)
print("ANÁLISIS")
print("="*70)

# Conflictos
conflicts = 0
for i in range(len(best_solution)):
    for j in range(i+1, len(best_solution)):
        if best_solution[i][0] == best_solution[j][0] and best_solution[i][1] == best_solution[j][1]:
            conflicts += 1

# Capacidad
capacity_violations = 0
for i, (block, room) in enumerate(best_solution):
    if courses[i][1] > rooms[room]:
        capacity_violations += 1

# Profesores
professor_conflicts = 0
for c1, c2 in same_professor:
    if best_solution[c1][0] == best_solution[c2][0]:
        professor_conflicts += 1

print(f"Conflictos de salón: {conflicts}")
print(f"Violaciones de capacidad: {capacity_violations}")
print(f"Conflictos de profesor: {professor_conflicts}")

if conflicts == 0 and capacity_violations == 0 and professor_conflicts == 0:
    print("\n✓ ¡SOLUCIÓN VÁLIDA! Todas las restricciones duras satisfechas.")
else:
    print("\n✗ Solución inválida. Hay restricciones violadas.")

---

# Preguntas de Análisis

## 1. Convergencia [5 puntos]
**¿En qué generación aproximadamente el algoritmo encontró una solución válida (sin violar restricciones duras)?**

*Tu respuesta:*



---

## 2. Función de Aptitud [5 puntos]
**¿Por qué es importante que las restricciones duras tengan penalizaciones más grandes (-50) que las blandas (-3 a -5)?**

*Tu respuesta:*



---

## 3. Operadores Genéticos [5 puntos]
**¿Qué sucedería si aumentáramos la tasa de mutación a 0.8 (80%)? ¿Mejoraría o empeoraría el algoritmo?**

*Tu respuesta:*



---

## 4. Tamaño de Población [5 puntos]
**Experimenta con poblaciones de 10, 30 y 50 individuos. ¿Qué observas en términos de velocidad de convergencia y calidad de la solución?**

*Tu respuesta:*



---

## Celda de Experimentación

Usa esta celda para probar diferentes parámetros:

In [ ]:
# Experimenta aquí

# Ejemplo: diferentes tamaños de población
# for size in [10, 30, 50]:
#     print(f"\n=== Población: {size} ===")
#     pop = create_population(size=size)
#     best = evolve(pop, generations=50, verbose=False)
#     print(f"Mejor fitness: {fitness(best)}")


---

# Desafío Bonus [+10 puntos]

Implementa UNA de estas mejoras:

## Opción 1: Cruce de Dos Puntos
Modifica `crossover()` para usar dos puntos de corte.

## Opción 2: Mutación Inteligente  
Si detectas una violación específica, intenta corregirla:
- Conflicto de salón → cambiar salón
- Capacidad insuficiente → cambiar a salón más grande

## Opción 3: Visualización
Grafica la evolución del fitness usando matplotlib.

## Opción 4: Nueva Restricción
Añade penalización si dos cursos relacionados están muy cerca (mismo día).

---

In [ ]:
# Tu código bonus aquí


---

# Entrega

## Checklist:
- [ ] Función `fitness()` completa y funcionando
- [ ] Función `crossover()` completa y funcionando  
- [ ] Función `mutate()` completa y funcionando
- [ ] Todas las celdas ejecutadas sin errores
- [ ] Preguntas de análisis respondidas
- [ ] Nombre y código al inicio del notebook

## Criterios de Evaluación:
- Función de aptitud: 40 puntos
- Operador de cruce: 30 puntos
- Operador de mutación: 30 puntos
- Preguntas de análisis: 20 puntos (5 cada una)
- Código limpio y documentado: 10 puntos
- **Bonus (opcional): +10 puntos**

**Total: 130 puntos (máximo 140 con bonus)**

## Formato de Entrega:
1. Guarda el notebook como: `Lab_AG_NombreApellido.ipynb`
2. Ejecuta todas las celdas en orden (Kernel → Restart & Run All)
3. Verifica que no haya errores
4. Sube a la plataforma del curso

---

**¡Buena suerte!** 🚀